## Comparison of CNN + SVM Models for Mass Detection in Mammography Images

### Setup

First we install any necessary packages.

In [2]:
!pip install pydicom

In [3]:
!pip install --upgrade pip
!pip uninstall -y tensorflow PyHamcrest google-api-core google-cloud-core grpcio google-cloud-monitoring datalab google-cloud-bigquery google-auth six requests protobuf
!pip install --upgrade PyHamcrest google-api-core google-cloud-core grpcio datalab google-cloud-bigquery google-cloud-monitoring==0.31.1 google-auth==1.9 setuptools==41.0.0 six==1.13.0 requests==2.21.0 protobuf==3.8.0 tensorflow==2.1.0
!pip install --upgrade matplotlib pandas h5py sklearn tqdm bleach pillow seaborn 

!pip install saliency
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install sklearn
!pip install PyDrive

Requirement already up-to-date: pip in /usr/local/envs/py3env/lib/python3.5/site-packages (20.1)
Found existing installation: google-api-core 1.17.0
Uninstalling google-api-core-1.17.0:
  Successfully uninstalled google-api-core-1.17.0
Found existing installation: tensorflow 2.1.0
Uninstalling tensorflow-2.1.0:
  Successfully uninstalled tensorflow-2.1.0
Found existing installation: google-cloud-monitoring 0.31.1
Uninstalling google-cloud-monitoring-0.31.1:
  Successfully uninstalled google-cloud-monitoring-0.31.1
Found existing installation: protobuf 3.8.0
Uninstalling protobuf-3.8.0:
  Successfully uninstalled protobuf-3.8.0
Found existing installation: google-cloud-core 1.3.0
Uninstalling google-cloud-core-1.3.0:
  Successfully uninstalled google-cloud-core-1.3.0
Found existing installation: grpcio 1.28.1
Uninstalling grpcio-1.28.1:
  Successfully uninstalled grpcio-1.28.1
Found existing installation: six 1.13.0
Uninstalling six-1.13.0:
  Successfully uninstalled six-1.13.0
Found ex

ERROR: google-api-core 1.17.0 has requirement google-auth<2.0dev,>=1.14.0, but you'll have google-auth 1.9.0 which is incompatible.
ERROR: apache-airflow 1.9.0 has requirement bleach==2.1.2, but you'll have bleach 3.1.5 which is incompatible.
Requirement already up-to-date: matplotlib in /usr/local/envs/py3env/lib/python3.5/site-packages (3.0.3)
Requirement already up-to-date: pandas in /usr/local/envs/py3env/lib/python3.5/site-packages (0.25.3)
Requirement already up-to-date: h5py in /usr/local/envs/py3env/lib/python3.5/site-packages (2.10.0)
Requirement already up-to-date: sklearn in /usr/local/envs/py3env/lib/python3.5/site-packages (0.0)
Requirement already up-to-date: tqdm in /usr/local/envs/py3env/lib/python3.5/site-packages (4.46.0)
Requirement already up-to-date: bleach in /usr/local/envs/py3env/lib/python3.5/site-packages (3.1.5)
Requirement already up-to-date: pillow in /usr/local/envs/py3env/lib/python3.5/site-packages (7.1.2)
  Using cached seaborn-0.9.1-py2.py3-none-any.wh

  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.7.0
ERROR: Cannot uninstall 'seaborn'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [4]:
!pip install opencv-python-headless
!pip install Pillow

In [5]:
import cv2
print('imported cv2')

imported cv2


In [6]:
# import needed modules:

# general python utilities
import os
import platform
import random
import shutil
import sys
import time
import datetime
import functools
import itertools as it
import copy
import pydicom
#import cv2
from PIL import Image
import pickle
# recommended Python3 version >= 3.5
print('Python version: {}'.format(platform.sys.version))

# data-science & processing tools
import numpy as np
import pandas as pd
import sklearn.metrics
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import h5py


# progress bar
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm_notebook as tqdm

# plotting utilities
import seaborn as sns
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
import tensorflow.keras as K

# required TensorFlow version >= 2.0.0
tf_version = tf.__version__
print('TensorFlow version: {}'.format(tf_version))
assert int(tf_version[0]) >= 2, "Tensorflow version must be >= 2.0"

# seed random numbers for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)


print('finished imports')

Python version: 3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 21:41:56) 
[GCC 7.3.0]
TensorFlow version: 2.1.0
finished imports


Then we load the packages.

We load the preprocessed [INBreast](https://github.com/wentaozhu/deep-mil-for-whole-mammogram-classification/issues/12) dataset from Dropbox.

In [7]:
os.getcwd()


'/content/datalab'

In [8]:

label_set = np.load('/content/datalab/labels.npy')
print(label_set.shape)

(1827,)


In [9]:
#data_set = np.load('data.npy')
!wget -O data_set https://www.dropbox.com/s/7gfev1s9mb9ozdb/data.npy?dl=0 data_set

--2020-05-06 01:21:29--  https://www.dropbox.com/s/7gfev1s9mb9ozdb/data.npy?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7gfev1s9mb9ozdb/data.npy [following]
--2020-05-06 01:21:29--  https://www.dropbox.com/s/raw/7gfev1s9mb9ozdb/data.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc335c9ddeae20de5d35f5880aa9.dl.dropboxusercontent.com/cd/0/inline/A3IZFy-jyn2ueByB6-RplgVSJxB32xUiCi3PJGqshsPmxFg2o0hN9m_IZRLZCJZtPri8khKqFslZ9m9EStil9g76FL10hXqG9Wb5ashDmpY1oIuOu4MSb_jY6IEYuq_YWaU/file# [following]
--2020-05-06 01:21:30--  https://uc335c9ddeae20de5d35f5880aa9.dl.dropboxusercontent.com/cd/0/inline/A3IZFy-jyn2ueByB6-RplgVSJxB32xUiCi3PJGqshsPmxFg2o0hN9m_IZRLZCJZtPri8khKqFslZ9m9EStil9g76FL10hXqG9Wb5ashDmpY1oIuOu4MSb_jY6

In [11]:
data_set = np.load('data_set')
print(data_set.shape)

(1827, 224, 224, 3)


In [16]:
flattened = data_set.reshape(1827, 224*224,3)
flattened = np.sum(flattened, axis = 2)
print(flattened.shape)

(1827, 50176)


In [13]:
#with flattened images, compute the baseline for svm
flat_train,flat_test,label_train,label_test = train_test_split(flattened, label_set, test_size=.2, random_state=20)
print(flat_train.shape)
print(flat_test.shape)
print(label_train.shape)
print(label_test.shape)
svmachine2 = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 50176)
(366, 50176)
(1461,)
(366,)
svm-only setup done


In [14]:
svmachine2.fit(flat_train,label_train)
print('fitted')
label_predict= svmachine2.predict(flat_test)
print('prediction')
print(label_predict.shape, label_test.shape)

fitted
prediction
(366,) (366,)


In [15]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

In [16]:
print(label_predict) #example of predicted labels

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.

In [17]:
print(classification_report(label_test,label_predict))

             precision    recall  f1-score   support

        0.0       0.66      1.00      0.80       235
        1.0       0.93      0.10      0.18       131

avg / total       0.76      0.67      0.58       366



In [18]:
conf = np.array(confusion_matrix(label_test,label_predict, labels = [0,1]))
confusion = pd.DataFrame(conf, index = [0,1], columns=
                        [0,1])

In [19]:
confusion

,0,1
0,234,1
1,118,13


In [20]:
au_roc = roc_auc_score(label_test, label_predict)
print(au_roc)

0.547490661036219


In [59]:
!pip install -U protobuf==3.11.3

     |████████████████████████████████| 1.3 MB 2.8 MB/s eta 0:00:01
ERROR: google-api-core 1.17.0 has requirement google-auth<2.0dev,>=1.14.0, but you'll have google-auth 1.9.0 which is incompatible.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.8.0
    Uninstalling protobuf-3.8.0:
      Successfully uninstalled protobuf-3.8.0


In [38]:
#begin computing VGG16+SVM
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
model = VGG16(weights='imagenet', include_top=False)
print(model.summary())

vgg16_feature = model.predict(preprocess_input(data_set))

print(vgg16_feature.shape)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [41]:
#with vgg features, flatten and pass into SVM
vgg_flat = vgg16_feature.reshape(1827, 7*7 *512)
vgg_train,vgg_test,v_label_train,v_label_test = train_test_split(vgg_flat, label_set, test_size=.2, random_state=20)
print(vgg_train.shape)
print(vgg_test.shape)
print(v_label_train.shape)
print(v_label_test.shape)
svmachine_vgg = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 25088)
(366, 25088)
(1461,)
(366,)
svm-only setup done


In [42]:
#fit and predict
svmachine_vgg.fit(vgg_train,v_label_train)
print('fitted')

fitted


In [43]:
v_label_predict= svmachine_vgg.predict(vgg_test)
print('prediction')
print(v_label_predict.shape, v_label_test.shape)

prediction
(366,) (366,)


In [ ]:
print(vgg16_feature[1])

In [44]:
print(classification_report(v_label_test,v_label_predict))

             precision    recall  f1-score   support

        0.0       0.76      0.99      0.86       235
        1.0       0.95      0.44      0.60       131

avg / total       0.83      0.79      0.76       366



In [45]:
v_conf = np.array(confusion_matrix(v_label_test,v_label_predict, labels = [0,1]))
v_confusion = pd.DataFrame(v_conf, index = [0,1], columns=
                        [0,1])

In [46]:
v_confusion

,0,1
0,232,3
1,74,57


In [47]:
v_au_roc = roc_auc_score(v_label_test, v_label_predict)
print(v_au_roc)

0.7111742731849927


In [15]:
#VGG19 + SVM
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

model = VGG19(weights='imagenet', include_top=False)
print(model.summary())


vgg19_feature = model.predict(preprocess_input(data_set))

print(vgg19_feature.shape)

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [16]:
#with flattened vgg19 features passed into svm
vgg_19_flat = vgg19_feature.reshape(1827, 7*7 *512)
vgg_train,vgg_test,v_label_train,v_label_test = train_test_split(vgg_19_flat, label_set, test_size=.2, random_state=20)
print(vgg_train.shape)
print(vgg_test.shape)
print(v_label_train.shape)
print(v_label_test.shape)
svmachine_vgg = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 25088)
(366, 25088)
(1461,)
(366,)
svm-only setup done


In [17]:

svmachine_vgg.fit(vgg_train,v_label_train)
print('fitted')
v_label_predict= svmachine_vgg.predict(vgg_test)
print('prediction')
print(v_label_predict.shape, v_label_test.shape)


fitted
prediction
(366,) (366,)


NameError: name 'classification_report' is not defined

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
print(classification_report(v_label_test,v_label_predict))

             precision    recall  f1-score   support

        0.0       0.65      0.99      0.78       235
        1.0       0.67      0.03      0.06       131

avg / total       0.65      0.65      0.52       366



In [19]:
v_conf = np.array(confusion_matrix(v_label_test,v_label_predict, labels = [0,1]))
v_confusion = pd.DataFrame(v_conf, index = [0,1], columns=
                        [0,1])
v_confusion

,0,1
0,233,2
1,127,4


In [20]:
v_au_roc = roc_auc_score(v_label_test, v_label_predict)
print(v_au_roc)

0.5110118564235829


In [23]:
#inceptionv3 + svm
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
# create the base pre-trained model
# max pool for generalization and shape
model = InceptionV3(weights='imagenet', input_shape = (224, 224, 3), include_top=False, pooling = 'max')
res_feature = model.predict(data_set)


In [24]:
print(res_feature.shape)

(1827, 2048)


In [25]:
#pass flattened inceptionv3 features into svm
res_flat = res_feature
res_train,res_test,r_label_train,r_label_test = train_test_split(res_flat, label_set, test_size=.2, random_state=20)
print(res_train.shape)
print(res_test.shape)
print(r_label_train.shape)
print(r_label_test.shape)
svmachine_res = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 2048)
(366, 2048)
(1461,)
(366,)
svm-only setup done


In [26]:
svmachine_res.fit(res_train,r_label_train)
print('fitted')
r_label_predict= svmachine_res.predict(res_test)
print('prediction')
print(r_label_predict.shape, r_label_test.shape)


fitted
prediction
(366,) (366,)


In [27]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
print(classification_report(r_label_test,r_label_predict))

             precision    recall  f1-score   support

        0.0       0.79      0.95      0.86       235
        1.0       0.87      0.55      0.67       131

avg / total       0.82      0.81      0.80       366



In [29]:
r_conf = np.array(confusion_matrix(r_label_test,r_label_predict, labels = [0,1]))
r_confusion = pd.DataFrame(r_conf, index = [0,1], columns=
                        [0,1])
r_confusion

,0,1
0,224,11
1,59,72


In [30]:
r_au_roc = roc_auc_score(r_label_test, r_label_predict)
print(r_au_roc)

0.7514049049861946


In [31]:
print(model.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_573 (Conv2D)             (None, 111, 111, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_573 (BatchN (None, 111, 111, 32) 96          conv2d_573[0][0]                 
__________________________________________________________________________________________________
activation_571 (Activation)     (None, 111, 111, 32) 0           batch_normalization_573[0][0]    
_______________________________________________________________________________________

In [35]:
#with nasnet features
from keras.applications import nasnet
nasnetmobile = nasnet.NASNetMobile(weights='imagenet', input_shape = (224, 224, 3), include_top=False, pooling = 'max')
nas_feature = nasnetmobile.predict(data_set)
print(nas_feature.shape)

(1827, 1056)


In [37]:
nas_flat = nas_feature #already flattened
nas_train,nas_test,n_label_train,n_label_test = train_test_split(nas_flat, label_set, test_size=.2, random_state=20)
print(nas_train.shape)
print(nas_test.shape)
print(n_label_train.shape)
print(n_label_test.shape)
svmachine_nas = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 1056)
(366, 1056)
(1461,)
(366,)
svm-only setup done


In [38]:
svmachine_nas.fit(nas_train,n_label_train)
print('fitted')
n_label_predict= svmachine_nas.predict(nas_test)
print('prediction')
print(n_label_predict.shape, n_label_test.shape)
print(classification_report(n_label_test,n_label_predict))

fitted
prediction
(366,) (366,)
             precision    recall  f1-score   support

        0.0       0.75      0.97      0.85       235
        1.0       0.90      0.41      0.57       131

avg / total       0.80      0.77      0.75       366



In [43]:
n_conf = np.array(confusion_matrix(n_label_test,n_label_predict, labels = [0,1]))
n_confusion = pd.DataFrame(n_conf, index = [0,1], columns=
                        [0,1])
n_confusion
# n_au_roc = roc_auc_score(n_label_test, n_label_predict)
# print(n_au_roc)

,0,1
0,229,6
1,77,54


In [41]:
print(nasnetmobile.summary())

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_12[0][0]                   
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1041 (Activation)    (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

__________________________________________________________________________________________________
separable_conv_1_normal_right2_ (None, 28, 28, 44)   2332        activation_1094[0][0]            
__________________________________________________________________________________________________
separable_conv_1_normal_left5_2 (None, 28, 28, 44)   2332        activation_1096[0][0]            
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 28, 28, 44)   176         separable_conv_1_normal_left1_2[0
__________________________________________________________________________________________________
separable_conv_1_bn_normal_righ (None, 28, 28, 44)   176         separable_conv_1_normal_right1_2[
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 28, 28, 44)   176         separable_conv_1_normal_left2_2[0
__________

reduction_add3_reduce_8 (Add)   (None, 7, 7, 176)    0           reduction_left3_reduce_8[0][0]   
                                                                 separable_conv_2_bn_reduction_rig
__________________________________________________________________________________________________
add_12 (Add)                    (None, 7, 7, 176)    0           reduction_add_2_reduce_8[0][0]   
                                                                 reduction_left4_reduce_8[0][0]   
__________________________________________________________________________________________________
reduction_add4_reduce_8 (Add)   (None, 7, 7, 176)    0           separable_conv_2_bn_reduction_lef
                                                                 reduction_right5_reduce_8[0][0]  
__________________________________________________________________________________________________
cropping2d_12 (Cropping2D)      (None, 14, 14, 528)  0           zero_padding2d_12[0][0]          
__________

Total params: 4,269,716
Trainable params: 4,232,978
Non-trainable params: 36,738
__________________________________________________________________________________________________
None


In [45]:
#with mobilenetv2 features
from keras.applications.mobilenet_v2 import MobileNetV2
mobilenetv2 = MobileNetV2(input_shape= (224,224,3),  include_top=False, weights='imagenet', pooling='max')
den_feature = 0.743.predict(data_set)
print(den_feature.shape)

(1827, 1280)


In [46]:
den_flat = den_feature
den_train,den_test,d_label_train,d_label_test = train_test_split(den_flat, label_set, test_size=.2, random_state=20)
print(den_train.shape)
print(den_test.shape)
print(d_label_train.shape)
print(d_label_test.shape)
svmachine_den = SVC(kernel='rbf', probability=True, random_state=42)
print('svm-only setup done')

(1461, 1280)
(366, 1280)
(1461,)
(366,)
svm-only setup done


In [47]:
svmachine_den.fit(den_train,d_label_train)
print('fitted')
d_label_predict= svmachine_den.predict(den_test)
print('prediction')
print(d_label_predict.shape, d_label_test.shape)
print(classification_report(d_label_test,d_label_predict))

fitted
prediction
(366,) (366,)
             precision    recall  f1-score   support

        0.0       0.78      0.98      0.87       235
        1.0       0.94      0.50      0.66       131

avg / total       0.84      0.81      0.79       366



In [49]:
d_conf = np.array(confusion_matrix(d_label_test,d_label_predict, labels = [0,1]))
d_confusion = pd.DataFrame(d_conf, index = [0,1], columns=
                        [0,1])
d_confusion
d_au_roc = roc_auc_score(d_label_test, d_label_predict)
print(d_au_roc)

0.7433977586486924


In [50]:
print(mobilenetv2.summary())

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

block_5_depthwise_relu (ReLU)   (None, 28, 28, 192)  0           block_5_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_5_project (Conv2D)        (None, 28, 28, 32)   6144        block_5_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_5_project_BN (BatchNormal (None, 28, 28, 32)   128         block_5_project[0][0]            
__________________________________________________________________________________________________
block_5_add (Add)               (None, 28, 28, 32)   0           block_4_add[0][0]                
                                                                 block_5_project_BN[0][0]         
__________________________________________________________________________________________________
block_6_expand (Conv2D)         (None, 28, 28, 192)  6144        block_5_add[0][0]                
__________